In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install transformers datasets sklearn torch

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import os
# List contents of the combined_dataset folder
os.listdir('/content/drive/MyDrive/NER_final_project/data/')


['data_issue.txt',
 '.DS_Store',
 'data_dev.csv',
 'data_test.csv',
 'data_train.csv',
 'EDA.ipynb',
 'train',
 'dev',
 'test']

In [ ]:
# Install necessary libraries
!pip install transformers datasets sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import torch
from transformers import ElectraTokenizerFast, ElectraForTokenClassification, Trainer, TrainingArguments

# Load the ELECTRA tokenizer and model
tokenizer = ElectraTokenizerFast.from_pretrained("google/electra-base-discriminator", add_prefix_space=True)
model = ElectraForTokenClassification.from_pretrained("google/electra-base-discriminator", num_labels=3)

# Load your CSV dataset (change paths to your actual files)
train_data = pd.read_csv("/content/drive/MyDrive/NER_final_project/data/data_train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/NER_final_project/data/data_test.csv")

# Check unique labels in the dataset
print("Unique train labels:", train_data['label'].unique())
print("Unique test labels:", test_data['label'].unique())

# Define a mapping from labels to IDs
label2id = {
    "O": 0,  # Outside entity
    "B": 1,  # Beginning of entity
    "I": 2   # Inside entity
}

# Preprocess the data (tokenization and label alignment)
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples['token'],
        truncation=True,
        padding='max_length',  # Pad sequences to a fixed length
        max_length=128,        # Define a fixed length
        is_split_into_words=True
    )

    labels = examples['label']
    word_ids = tokenized_inputs.word_ids()

    # Align labels to tokenized inputs
    new_labels = []
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            new_labels.append(-100)  # Ignore padding tokens
        elif word_idx != previous_word_idx:
            new_labels.append(label2id[labels[word_idx]])
        else:
            new_labels.append(-100)  # Ignore subword tokens
        previous_word_idx = word_idx

    # Check for length mismatch between tokens and labels
    assert len(new_labels) == len(tokenized_inputs["input_ids"]), \
        f"Mismatch between tokenized input and label length: {len(new_labels)} != {len(tokenized_inputs['input_ids'])}"

    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

# Convert datasets to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)

# Apply preprocessing
train_dataset = train_dataset.map(lambda x: tokenize_and_align_labels({
    'token': x['token'].split(),
    'label': x['label'].split()
}), remove_columns=['token', 'label'])
test_dataset = test_dataset.map(lambda x: tokenize_and_align_labels({
    'token': x['token'].split(),
    'label': x['label'].split()
}), remove_columns=['token', 'label'])

# Compute metrics
def compute_metrics(p):
    predictions = torch.tensor(p.predictions)
    labels = torch.tensor(p.label_ids)
    predictions = torch.argmax(predictions, axis=-1).flatten()
    labels = labels.flatten()
    mask = labels != -100
    predictions = predictions[mask]
    labels = labels[mask]
    accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
    precision = precision_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='weighted')
    recall = recall_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='weighted')
    f1 = f1_score(labels.cpu().numpy(), predictions.cpu().numpy(), average='weighted')
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none'
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model on the test dataset
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)

# Prediction on a new sentence
example_sentence = ["Systems", "based", "on", "synchronous", "grammars", "and", "tree"]

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize the sentence and move inputs to the correct device
inputs = tokenizer(example_sentence, return_tensors="pt", is_split_into_words=True)
inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to the device

# Make prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to predicted labels
predicted_ids = torch.argmax(logits, axis=-1)
predicted_labels = [list(label2id.keys())[id] for id in predicted_ids[0].tolist()]


print("Predicted labels:", predicted_labels)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

...

Map:   0%|          | 0/3899 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.440700,0.409902,0.832521,0.822681,0.832521,0.826883
2,0.383300,0.410637,0.836881,0.826170,0.836881,0.830466


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.440700,0.409902,0.832521,0.822681,0.832521,0.826883
2,0.383300,0.410637,0.836881,0.826170,0.836881,0.830466
3,0.348400,0.438187,0.835342,0.830138,0.835342,0.832149


Evaluation results: {'eval_loss': 0.4381870925426483, 'eval_accuracy': 0.8353423954860221, 'eval_precision': 0.8301375452840037, 'eval_recall': 0.8353423954860221, 'eval_f1': 0.8321487511369142, 'eval_runtime': 29.5606, 'eval_samples_per_second': 131.898, 'eval_steps_per_second': 8.254, 'epoch': 3.0}
Predicted labels: ['I', 'I', 'I', 'O', 'B', 'I', 'I', 'I', 'I', 'I', 'O', 'B', 'I']


In [ ]:
# Save the model (PyTorch) using Hugging Face's save_pretrained method
model.save_pretrained("./fine_tuned_electra_model")
tokenizer.save_pretrained("./fine_tuned_electra_model")  # Save the tokenizer too
print("Model and tokenizer saved.")



Model and tokenizer saved.


In [ ]:
from transformers import ElectraForTokenClassification, ElectraTokenizerFast

# Load the saved model and tokenizer
model = ElectraForTokenClassification.from_pretrained("./fine_tuned_electra_model")
tokenizer = ElectraTokenizerFast.from_pretrained("./fine_tuned_electra_model")
print("Model and tokenizer loaded.")



Model and tokenizer loaded.


In [ ]:
# Input sentence for testing
sentence = "This paper aim to solve machine translation Tagging problem using neural networks."

# Tokenize the sentence
inputs = tokenizer(sentence.split(), return_tensors="pt", is_split_into_words=True)

# Move the model and inputs to the correct device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}


In [ ]:
# Make the prediction
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Convert logits to predicted labels
predicted_ids = torch.argmax(logits, axis=-1)

# Map predicted labels back to actual label names
label2id = {
    "O": 0,  # Outside entity
    "B": 1,  # Beginning of entity
    "I": 2   # Inside entity
}
id2label = {v: k for k, v in label2id.items()}  # Reverse the label2id mapping

predicted_labels = [id2label[id] for id in predicted_ids[0].tolist()]

print("Predicted labels:", predicted_labels)
# Tokenize sentence again to align with labels
tokens = sentence.split()

# Display tokens and their corresponding predicted labels
for token, label in zip(tokens, predicted_labels):
    print(f"{token}: {label}")



Predicted labels: ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'B', 'I', 'I', 'O', 'O', 'B', 'I', 'O', 'O']
This: O
paper: O
aim: O
to: O
solve: O
machine: O
translation: B
Tagging: B
problem: I
using: I
neural: O
networks.: O


In [ ]:
model.save_pretrained("/content/drive/MyDrive/fine_tuned_electra_model")
tokenizer.save_pretrained("/content/drive/MyDrive/fine_tuned_electra_model")


('/content/drive/MyDrive/fine_tuned_electra_model/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_electra_model/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_electra_model/vocab.txt',
 '/content/drive/MyDrive/fine_tuned_electra_model/added_tokens.json',
 '/content/drive/MyDrive/fine_tuned_electra_model/tokenizer.json')

In [ ]:
import os
print(os.listdir('/content/drive/MyDrive/fine_tuned_electra_model'))


['config.json', 'model.safetensors', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.txt', 'tokenizer.json']
